In [1]:
import os
import functools

import wallycore as wally

h2b = wally.hex_to_bytes
b2h = wally.hex_from_bytes

# asset_ids are reversed like txhashes
BITCOIN = h2b("5ac9f65c0efcc4775e0baec4ec03abdde22473cd3cf33c0419ca290e0751b225")[::-1]

secret_key = h2b("eb412c12e06023dd05bb568375ea08238ea8bceb369177a4a2ad1309080a3138")
pubkey = wally.ec_public_key_from_private_key(secret_key)
print("pubkey: {}".format(b2h(pubkey)))

# prefix "235" from: https://github.com/Blockstream/gdk/blob/master/src/network_parameters.cpp#L111 
address = wally.base58_from_bytes(bytearray([235]) + wally.hash160(pubkey), wally.BASE58_FLAG_CHECKSUM)
print("my address: {}".format(address))

pubkey: 026d1b787eb102b4f8fe2627295f57119c26bc4cf019daab3c1fd20c270aa7746c
my address: 2dh7NbLFMyZA1Nxe6AYAB7MPr87BpWvCkqs


In [2]:
# input data
in_txhash = h2b("8e22a313de4c195c489e3a80376787050d9f361ec70389c53586ff55d4fa2f4a")[::-1] # reversed
in_vout = 0
input_scriptpubkey = bytearray([wally.OP_DUP, wally.OP_HASH160, 20]) + wally.hash160(pubkey) + bytearray([wally.OP_EQUALVERIFY, wally.OP_CHECKSIG])

# fee
fee = 300

# output data
out_script = bytearray([wally.OP_RETURN, 0x00])
out_amount = int(1e8) - fee

In [3]:
# create the tx, one input and two outputs (one is for the fee because it's explicit)
tx = wally.tx_init(
    2, # version
    0, # locktime
    1, # allocate one input
    2, # allocate two outputs
)

wally.tx_add_elements_raw_input(
    tx,
    in_txhash, # input txhash (reverted)
    in_vout, # input vout
    0xffffffff, # sequence number
    None, # scriptsig, we will fill this later
    None, # witness, null since it's a legacy tx
    None, # nonce, used for blinded txs
    None, # entropy, used for asset issuances
    None, # issuance_amount
    None, # inflation keys
    None, # issuance_amount_rangeproof
    None, # inflation_keys_rangeproof
    None, # pegin_witness
    0, # wally flag, must be zero
)

wally.tx_add_elements_raw_output(
    tx,
    out_script, # script we are paying
    bytearray([0x01]) + BITCOIN, # asset_tag, we add 0x01 to show that it's explicit and not blinded
    wally.tx_confidential_value_from_satoshi(out_amount), # value, the function will add a prefix to show that it's explicit
    None, # nonce, used for blinded txs
    None, # surjection_proof, used for blinded txs
    None, # range_proof, used for blinded txs
    0 # wally flag, must be zero
)

wally.tx_add_elements_raw_output(
    tx,
    None, # fees have no script
    bytearray([0x01]) + BITCOIN, # asset_tag, we add 0x01 to show that it's explicit and not blinded
    wally.tx_confidential_value_from_satoshi(fee), # value, the function will add a prefix to show that it's explicit
    None, # nonce, used for blinded txs
    None, # surjection_proof, used for blinded txs
    None, # range_proof, used for blinded txs
    0 # wally flag, must be zero
)

In [4]:
print("unsigned tx: {}".format(wally.tx_to_hex(tx, 0)))

unsigned tx: 0200000000014a2ffad455ff8635c58903c71e369f0d05876737803a9e485c194cde13a3228e0000000000ffffffff020125b251070e29ca19043cf33ccd7324e2ddab03ecc4ae0b5e77c4fc0e5cf6c95a010000000005f5dfd400026a000125b251070e29ca19043cf33ccd7324e2ddab03ecc4ae0b5e77c4fc0e5cf6c95a01000000000000012c000000000000


In [5]:
sighash = wally.tx_get_elements_signature_hash(
    tx,
    0, # sign input 0
    input_scriptpubkey, # scriptpubkey of the input we are signing
    None, # value of the input, only used for segwit txs (see bip143)
    wally.WALLY_SIGHASH_ALL, # sighash
    0, # flags, can be 0 or WALLY_TX_FLAG_USE_WITNESS for bip143 signatures
)

# make an ECDSA sig and also try to get a shorter R (saves one byte in the final tx almost for free)
compact_sig = wally.ec_sig_from_bytes(secret_key, sighash, wally.EC_FLAG_ECDSA | wally.EC_FLAG_GRIND_R)
der_sig = wally.ec_sig_to_der(compact_sig)

# size of the following element, der_sig + sighash flag + size of the following element (pubkey) + pubkey
script_sig = bytearray([len(der_sig) + 1]) + der_sig + bytearray([wally.WALLY_SIGHASH_ALL]) + bytearray([len(pubkey)]) + pubkey

# set the scriptsig for input 0
wally.tx_set_input_script(tx, 0, script_sig)

print("signed tx: {}".format(wally.tx_to_hex(tx, 0)))

signed tx: 0200000000014a2ffad455ff8635c58903c71e369f0d05876737803a9e485c194cde13a3228e000000006a473044022000fc76dcebc54bf707a02c755e0f031fbdf3858fc7ecc6f5f0757a5cec37fc500220145ab0565e84e5bbd36617930baab78d61ddcaf8765fa541a50f216811f2af9a0121026d1b787eb102b4f8fe2627295f57119c26bc4cf019daab3c1fd20c270aa7746cffffffff020125b251070e29ca19043cf33ccd7324e2ddab03ecc4ae0b5e77c4fc0e5cf6c95a010000000005f5dfd400026a000125b251070e29ca19043cf33ccd7324e2ddab03ecc4ae0b5e77c4fc0e5cf6c95a01000000000000012c000000000000
